In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from pandas.testing import assert_frame_equal
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as ss
p1 = Path.cwd() / 'back_data'

In [2]:
# Pandas 객체 병합은 append() 메서드(곧 deprecate 예정), concat() 함수, join() 메서드, merge() 메서드 등 존재

In [3]:
names = pd.read_csv(p1 / 'names.csv')
names

,Name,Age
0,Cornelia,70
1,Abbas,69
2,Penelope,4
3,Niko,2


In [4]:
# loc 속성을 통해 새로운 행 추가 가능(단, 원시 DataFrame 변형 발생) -> 리스트, 딕셔너리, Series 등 활용 가능
# ex. names.loc[len(name)] = pd.Series({'Name':'Dean', 'Age':32}))

In [7]:
# append() 메서드를 활용할 경우 ignore_index=True 인자 꼭 전달해야
(names
.append({'Name':'Aria', 'Age':1}, ignore_index=True)
# 그게 아니라면 전달하는 Series에서 name 파라미터를 필수로 활용해야
# Series는 오직 하나의 열만 있는 개념. 즉 index와 columns가 아님 -> 그래서 name 파라미터를 쓴다고 생각해야
.append(pd.Series({'Name':'Zach', 'Age':3}, name=len(names)))
)

C:\Users\admin\AppData\Local\Temp\ipykernel_19236\1346535784.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  (names
C:\Users\admin\AppData\Local\Temp\ipykernel_19236\1346535784.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  (names


,Name,Age
0,Cornelia,70
1,Abbas,69
2,Penelope,4
3,Niko,2
4,Aria,1
4,Zach,3


In [8]:
bball_16 = pd.read_csv(p1 / 'baseball16.csv')
bball_16.head()

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,...,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
0,altuvjo01,2016,1,HOU,AL,161,640,108,216,42,...,96.0,30.0,10.0,60,70.0,11.0,7.0,3.0,7.0,15.0
1,bregmal01,2016,1,HOU,AL,49,201,31,53,13,...,34.0,2.0,0.0,15,52.0,0.0,0.0,0.0,1.0,1.0
2,castrja01,2016,1,HOU,AL,113,329,41,69,16,...,32.0,2.0,1.0,45,123.0,0.0,1.0,1.0,0.0,9.0
3,correca01,2016,1,HOU,AL,153,577,76,158,36,...,96.0,13.0,3.0,75,139.0,5.0,5.0,0.0,3.0,12.0
4,gattiev01,2016,1,HOU,AL,128,447,58,112,19,...,72.0,2.0,1.0,43,127.0,6.0,4.0,0.0,5.0,12.0


In [12]:
# to_dict() 메서드를 통해 예제 행을 딕셔너리 형태로 추출
data_dict = bball_16.iloc[0].to_dict()
# dictionary comprehension 이용하여 딕셔너리의 값을 다 제거(문자열을 경우 공백, 숫자일 경우 np.nan 형태)
# isinstance() 활용하거나 혹은 type() 함수 활용도 가능 (if type(v) == str)
new_data_dict = {k:'' if isinstance(v, str) else np.nan for k, v in data_dict.items()}
new_data_dict

{'playerID': '',
 'yearID': nan,
 'stint': nan,
 'teamID': '',
 'lgID': '',
 'G': nan,
 'AB': nan,
 'R': nan,
 'H': nan,
 '2B': nan,
 '3B': nan,
 'HR': nan,
 'RBI': nan,
 'SB': nan,
 'CS': nan,
 'BB': nan,
 'SO': nan,
 'IBB': nan,
 'HBP': nan,
 'SH': nan,
 'SF': nan,
 'GIDP': nan}

In [ ]:
# DataFrame에 단일 행을 추가하는 것은 비싼 작업 -> 단일 행 추가로 루프(순환)을 진행한다면 잘못된 행위
# append() 메서드 활용할 경우 하나의 행씩 추가하는 게 아니라, 그냥 전체 행의 리스트 자체를 append() 메서드에 삽입
# 내부적으로 Pandas는 Series 리스트를 단일 DataFrame으로 변환한 다음 데이터를 추가